In [ ]:
import os
import subprocess
import pandas as pd
import shutil
import tqdm
import glob
import pytest
import numpy as np
import matplotlib.pyplot as plt
import pydqc.data_compare as dc 
from joblib import Parallel, delayed



# Datenpreparation für das ML-Modell


##### Zunächst muss ich die Daten aus dem Source-Ordner der Bilder in die entsprechenden Ordnerverteilen um Beispielsweise in PyTorch mit der Funktion Imagefolder auch arbeiten zu können.
##### Hierfür schreiben ich zunächst eine Funktion. 

In [ ]:
csv_path=r"C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\data\attribute_list_labels\list_attr_celeba.csv"


def get_image_paths(source_path):
    image_formats = ['*.jpg', '*.png', '*.gif', '*.jpeg']
    image_paths = []
    for format in image_formats:
        image_paths.extend(glob.glob(os.path.join(source_path, format)))
    return image_paths

def replace_image_paths(csv_path, new_image_paths, image_id="image_id"):
    df = pd.read_csv(csv_path)
    df[image_id] = new_image_paths
    df.to_csv(csv_path, index=False)


def test_data_quality(csv_path):
    df = pd.read_csv(csv_path)

    # Überprüfe auf fehlende Werte
    assert df['image_id'].isnull().sum() == 0, "Es gibt fehlende Werte in der Spalte 'image_id'"

    # Überprüfe auf Duplikate
    assert df.duplicated().sum() == 0, "Es gibt Duplikate in der Daten"

    # Überprüfe, ob alle Werte in der Spalte 'image_id' gültige Bildpfade sind
    assert all(df['image_id'].apply(os.path.isfile)), "Nicht alle Werte in der Spalte 'image_id' sind gültige Bildpfade"


def check_missing_values(csv_path):
    df = pd.read_csv(csv_path)
    missing_values = df.isnull().any()
    return missing_values

def test_outliers_all_columns(csv_path):
    df = pd.read_csv(csv_path)
    for column_name in df.columns:
        if np.issubdtype(df[column_name].dtype, np.number):  # Überprüfe, ob die Spalte numerisch ist
            z_scores = np.abs((df[column_name] - df[column_name].mean()) / df[column_name].std())
            assert not any(z_scores > 3), f"Es gibt Ausreißer in der Spalte '{column_name}'"

def test_balance(csv_path, column_name):
    df = pd.read_csv(csv_path)
    counts = df[column_name].value_counts()
    assert abs(counts[-1] - counts[1]) < 0.1 * len(df), f"Die Spalte '{column_name}' ist unausgeglichen. Anzahl von -1: {counts[-1]}, Anzahl von 1: {counts[1]}"

def test_balance_all_columns(csv_path):
    df = pd.read_csv(csv_path)
    imbalance_report = []

    for column_name in df.columns:
        if np.issubdtype(df[column_name].dtype, np.number):  # Überprüfe, ob die Spalte numerisch ist
            counts = df[column_name].value_counts()
            if abs(counts.get(-1, 0) - counts.get(1, 0)) >= 0.1 * len(df):
                imbalance_report.append(f"Die Spalte '{column_name}' ist unausgeglichen. Anzahl von -1: {counts.get(-1, 0)}, Anzahl von 1: {counts.get(1, 0)}")

    assert not imbalance_report, "Es gibt unausgeglichene Spalten:\n" + "\n".join(imbalance_report)

def plot_balance_all_columns(csv_path):
    df = pd.read_csv(csv_path)

    for column_name in df.columns:
        if np.issubdtype(df[column_name].dtype, np.number):  # Überprüfe, ob die Spalte numerisch ist
            counts = df[column_name].value_counts()
            counts.plot(kind='bar', title=f"Verteilung der Werte in der Spalte '{column_name}'")
            plt.show()


# check_missing_values(csv_path=r"C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\data\attribute_list_labels\list_attr_celeba.csv")
# test_data_quality(csv_path=r"C:\Users\busse\Bachelorarbeit\CICD-Pipeline-Gender-Recognition\data\attribute_list_labels\list_attr_celeba.csv")
plot_balance_all_columns(csv_path=csv_path)
test_balance_all_columns(csv_path=csv_path)



In [ ]:
def detect_outliers_iqr_all_columns(csv_path):
    df = pd.read_csv(csv_path)
    outliers_report = []

    for column_name in df.columns:
        if np.issubdtype(df[column_name].dtype, np.number):  # Überprüfe, ob die Spalte numerisch ist
            Q1 = df[column_name].quantile(0.25)
            Q3 = df[column_name].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df[(df[column_name] < Q1 - 1.5 * IQR) | (df[column_name] > Q3 + 1.5 * IQR)]
            if not outliers.empty:
                outliers_report.append(f"Die Spalte '{column_name}' hat Ausreißer. Anzahl: {len(outliers)}")

    return "\n".join(outliers_report)

detect_outliers_iqr_all_columns(csv_path=csv_path)

In [ ]:
def check_duplicates(csv_path):
    df = pd.read_csv(csv_path)
    duplicates = df.duplicated()
    if duplicates.any():
        print(f"Es gibt {duplicates.sum()} Duplikate in den Daten.")
    else:
        print("Es gibt keine Duplikate in den Daten.")

check_duplicates(csv_path=csv_path)

In [ ]:
def check_null_values(csv_path):
    df = pd.read_csv(csv_path)
    null_values = df.isnull().sum()
    if null_values.any():
        print(f"Es gibt Nullwerte in den Daten:\n{null_values}")
    else:
        print("Es gibt keine Nullwerte in den Daten.")

check_null_values(csv_path=csv_path)

In [ ]:
from great_expectations.data_context import DataContext

context = DataContext('great_expectations.yml')

# Load a batch of data
batch = context.get_batch('my_datasource', 'my_expectation_suite', 'my_data')

# Check if the data meets your expectations
results = context.run_validation_operator('action_list_operator', assets_to_validate=[batch])

# Print the results
print(results)
